In [ ]:
pip install -q tensorflow==2.4.1

In [ ]:
pip install -q tensorflow-quantum

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit


### Data Read and Pre-processing

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
df_x = pd.read_csv('/content/drive/My Drive/zadid/X_train2.txt',names=[ str(i) for i in range(1,14)])
df_y = pd.read_csv('/content/drive/My Drive/zadid/y_train2.txt',names=['label'])

df_x_test = pd.read_csv('/content/drive/My Drive/zadid/X_test2.txt',names=[ str(i) for i in range(1,14)])
df_y_test = pd.read_csv('/content/drive/My Drive/zadid/y_test2.txt',names=['label'])


df_x = df_x.abs()
df_x_test = df_x_test.abs()

In [ ]:
min_max_scaler = MinMaxScaler()
scaler = StandardScaler()
df_x = pd.DataFrame(scaler.fit_transform(df_x), columns=df_x.columns)
df_x = pd.DataFrame(min_max_scaler.fit_transform(df_x), columns=df_x.columns)

df_x_test = pd.DataFrame(scaler.fit_transform(df_x_test), columns=df_x.columns)
df_x_test = pd.DataFrame(min_max_scaler.fit_transform(df_x_test), columns=df_x.columns)

### Build CAN image dataset

In [ ]:
x = []
y = []

np_x = df_x.to_numpy()
np_y = df_y.to_numpy()


print ('processing train data...')
for i in range(len(np_x)-13):
    if(i%13==0):
      img = np_x[i:i+13,:].reshape(13,13,1)
      img = np.resize(img,(4,4,1))
      label = 1 if 1 in np_y[i:i+13,:] else 0
      x.append(img)
      y.append(label)

x_train = np.array(x)
y_train = np.array(y)

In [ ]:
x = np.array(x_train)
y = np.array(y_train)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


### Train a convolution neural network (CNN)

In [ ]:
def create_full_classical_model():
    # A simple model based off LeNet 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(13,13,1)))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(16))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_full_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

EPOCHS = 100
BATCH_SIZE = 8

fair_history = model.fit(x_train,
          y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(x_test, y_test))

fair_nn_results = model.evaluate(x_test, y_test)

In [ ]:
plt.plot(fair_history.history['accuracy'])
plt.plot(fair_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(fair_history.history['loss'])
plt.plot(fair_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from sklearn.preprocessing import minmax_scale

print (x_test[0].shape)
layer_name = model.layers[-2].name
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x_test[0][None, :, :, :])

foo_norm = minmax_scale(intermediate_output, feature_range=(0,1), axis=1)
print (foo_norm)


In [ ]:
import sys
def show_progress(count, total, status=''):

    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '#' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [ ]:
x_trans = []
y_trans = []
total = len(x_train)
for i in range(len(x_train)):
  x_trans.append(x_train[i])
  y_trans.append(y_train[i])

print ('Done')

In [ ]:
x_trans_test = []
y_trans_test = []

for i in range(len(x_test)):
  x_trans_test.append(x_test[i])
  y_trans_test.append(y_test[i])

print ('Done')

In [ ]:
x_trans = np.array(x_trans)
y_trans = np.array(y_trans)

print (x_trans.shape)
print (y_trans.shape)

In [ ]:
x_trans_test = np.array(x_trans_test)
y_trans_test = np.array(y_trans_test)

print (x_trans_test.shape)
print (y_trans_test.shape)

### Perform Quantum Encoding

In [ ]:
per = 1.0
x_train_nums = int(x_trans.shape[0]*per)
x_test_nums = int(x_trans_test.shape[0]*1.0)

In [ ]:
x_train = x_trans[:x_train_nums]
x_test = x_trans_test[:x_test_nums]
y_train = y_trans[:x_train_nums]
y_test = y_trans_test[:x_test_nums]

x_train_small = tf.image.resize(x_train, (4,4)).numpy()
x_test_small = tf.image.resize(x_test, (4,4)).numpy()

In [ ]:
THRESHOLD = 0.5

x_train_bin = np.array(x_train_small > THRESHOLD, dtype=np.float32)
x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

print (x_train_bin.shape)
print (x_test_bin.shape)

In [ ]:
import random as random

idx = random.randint(0,len(x_train_bin))
print('index:', idx , ' label : ', y_train[idx])


plt.imshow(x_train_bin[idx,:,:,0])
plt.colorbar()

### Build Quantum Circuit

In [ ]:
def convert_to_circuit(image):
    """Encode truncated classical image into quantum datapoint."""
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
      if value:
        circuit.append(cirq.X(qubits[i])) 
    return circuit

In [ ]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)

In [ ]:
class CircuitLayerBuilder():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout
    
    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

In [ ]:
def create_quantum_model():
    """Create a QNN model circuit and readout operation to go along with it."""
    data_qubits = cirq.GridQubit.rect(4, 4)  # a 4x4 grid.
    readout = cirq.GridQubit(-1, -1)         # a single qubit at [-1,-1]
    circuit = cirq.Circuit()
    
    # Prepare the readout qubit.
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))
    
    builder = CircuitLayerBuilder(
        data_qubits = data_qubits,
        readout=readout)

    # Then add layers (experiment by adding more).
    builder.add_layer(circuit, cirq.XX, "xx1")
    builder.add_layer(circuit, cirq.ZZ, "zz1")
    builder.add_layer(circuit, cirq.XX, "xx2")
    builder.add_layer(circuit, cirq.ZZ, "zz2")

    builder.add_layer(circuit, cirq.XX, "xx3")
    builder.add_layer(circuit, cirq.ZZ, "zz3")

    # Finally, prepare the readout qubit.
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

# create the circuit 
model_circuit, model_readout = create_quantum_model()    

In [ ]:
model_circuit, model_readout = create_quantum_model()

In [ ]:
import sympy

In [ ]:
SVGCircuit(model_circuit)

In [ ]:
# Build the Keras model.
model = tf.keras.Sequential([
    # The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    # The PQC layer returns the expected value of the readout gate, range [-1,1].
    tfq.layers.PQC(model_circuit, model_readout),
])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

In [ ]:
y_train_hinge = 2.0*y_train-1.0
y_test_hinge = 2.0*y_test-1.0

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [ ]:
model.compile(
    loss=tf.keras.losses.Hinge(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[hinge_accuracy])


In [ ]:
print(model.summary())
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 8

NUM_EXAMPLES = len(x_train_tfcirc)

In [ ]:
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_hinge[:NUM_EXAMPLES]

### Train Qunantum NN

In [ ]:
qnn_history = model.fit(
      x_train_tfcirc_sub, y_train_hinge_sub,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      verbose=1,
      validation_data=(x_test_tfcirc, y_test_hinge))

In [ ]:
qnn_results = model.evaluate(x_test_tfcirc, y_test_hinge)